In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=20, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Trial 468 Complete [03h 14m 56s]
val_weightedf2score: 0.694920539855957

Best val_weightedf2score So Far: 0.6974008679389954
Total elapsed time: 9d 19h 01m 11s

Search: Running Trial #469

Value             |Best Value So Far |Hyperparameter
True              |True              |class_weight
2017              |2017_2018_2019    |training_years
l1l2              |l1l2              |kernel_regularizer
0.5               |0.1               |spatial_dropout
leaky_relu        |relu              |activation
4                 |4                 |pool_size
0.1               |0.5               |dropout
True              |True              |bias_initializer
0.001             |0.0001            |learning_rate
binary_crossent...|binary_crossent...|loss_function
34                |34                |tuner/epochs
0                 |12                |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/34
2930/3550 ━━━━━━━━

In [ ]:
# best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# best_hp.values['units'] = 128
# best_hp.values['dropout'] = 0.3

# model = tuner.hypermodel.build(best_hp)
# model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"